# Travel Planner Agent

This project is a multi-agent, comprehensive travel planning system that helps users plan trips by providing flight options, hotel recommendations, and detailed daily itineraries. The system uses a modular architecture with three specialized agents working together to create a complete travel plan.

## Architecture
The project follows an agent-based architecture with three specialized agents:

1. Flight Agent : Searches for flight options between origin and destination airports. It uses the Aviation Stack API as the primary data source with OpenAI as a fallback.
2. Hotel Agent : Finds hotel accommodations at the destination. It uses OpenAI to generate realistic hotel options with details like name, address, rating, price range, and amenities.
3. Itinerary Agent : Creates a day-by-day travel plan based on the selected flights, hotels, and destination. It uses OpenAI to generate activities and schedules tailored to the destination.

## APIs Used
The system integrates with two external APIs:

1. Aviation Stack API : Provides real-time flight data including schedules, airlines, and flight numbers. Used by the Flight Agent to find actual flight options.
2. OpenAI API : Used for generating realistic travel data when real API data is unavailable. All three agents use OpenAI as either their primary data source or as a fallback mechanism.
## Workflow
1. The user provides origin, destination, date, and trip duration
2. The Flight Agent finds available flights
3. The Hotel Agent finds accommodation options
4. The Itinerary Agent creates a daily schedule
5. The TravelPlanner combines all this information into a comprehensive trip plan
6. Results are displayed in the console and saved to an output file
The main function demonstrates this workflow with a sample trip from SFO (San Francisco) to Tokyo for 5 days starting on September 15, 2024.

This modular design makes the system flexible and maintainable, allowing for easy updates or replacements of individual components without affecting the rest of the system.

In [4]:
!pip install python-dotenv --upgrade openai

   ---------------------------------------- 0.0/599.1 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/599.1 kB ? eta -:--:--
   ---------------------------------------- 599.1/599.1 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.1
    Uninstalling python-dotenv-1.0.1:
      Successfully uninstalled python-dotenv-1.0.1
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


### 1. Imports

This cell imports all necessary libraries for the travel planner

In [9]:
import requests
import os
import json # parses and generates JSON data for API communication
import re
import datetime # manages dates for trip planning and calculating flight durations
import random #generates random prices when API data doesn't include pricing
from typing import List, Dict, Any # provides type hints for better code readability and IDE support
import functools
from typing import Callable, TypeVar, cast, Any, Dict, List, Optional

### 2. MCP Class

The MCP class implements a standardized communication protocol for AI agents. It provides a decorator-based approach to enhance functions with structured context information.

#### Key Components:
1. MCP Class : A utility class that standardizes how agents communicate with language models.
2. @staticmethod tool Decorator : The main feature of this class that:
   
   - Takes a task description template and required response format
   - Wraps functions to automatically generate structured context
   - Injects this context into the function call
3. Context Generation Process :
   
   - Extracts function arguments using Python's inspect module
   - Formats the task description with the provided arguments
   - Creates a standardized context dictionary with agent name, task, format, and parameters
   - Passes this context to the wrapped function as mcp_context

In [10]:
class MCP:
    """Model Context Protocol module for standardized agent communication."""
    
    @staticmethod
    def tool(
        task_description: str,
        required_format: str = "json"
    ) -> Callable:
        """
        Decorator for functions that use the Model Context Protocol.
        
        Args:
            task_description (str): Template string describing the task
            required_format (str): Expected response format
            
        Returns:
            Callable: Decorated function
        """
        def decorator(func: Callable) -> Callable:
            @functools.wraps(func)
            def wrapper(self, *args, **kwargs):
                # Get function arguments
                import inspect
                sig = inspect.signature(func)
                bound_args = sig.bind(self, *args, **kwargs)
                bound_args.apply_defaults()
                
                # Create context from arguments
                arg_dict = dict(bound_args.arguments)
                arg_dict.pop('self', None)  # Remove 'self' from context
                
                # Generate task description with arguments
                formatted_task = task_description.format(**arg_dict)
                
                # Create a context dictionary
                context = {
                    "agent": self.name,
                    "task": formatted_task,
                    "format": required_format,
                    "parameters": arg_dict
                }
                
                # Call the original function with the context
                return func(self, *args, **kwargs, mcp_context=context)
            return wrapper
        return decorator

# Create an instance of the MCP module
mcp = MCP()

### 2. BaseAgent Class 
This class provides core functionality for making API calls to OpenAI's language models with standardized formatting and error handling.

#### Key Components:
1. Initialization :
   
   - Takes a name for the agent and an optional API key
   - Falls back to environment variables if no API key is provided

2. OpenAI API Integration :
   
   - The call_openai method handles communication with OpenAI's API
   - Supports both standard prompts and structured Model Context Protocol (MCP) formatting

3. MCP Implementation :
   
   - When an MCP context is provided, it formats the prompt with a standardized header
   - Includes agent name, task description, expected response format, and parameters
   - This structured approach helps the language model understand exactly what's expected
   
4. JSON Handling :
   
   - The _extract_json method attempts to parse JSON from the model's response
   - Includes fallback mechanisms to extract JSON from text if direct parsing fails
   - Returns error information if parsing fails completely
This class is designed to be extended by specialized agents (like FlightAgent, HotelAgent, etc.) that inherit these core capabilities while adding domain-specific functionality.

The MCP formatting is particularly important as it creates a consistent interface between different agents and the language model, making responses more predictable and easier to process.

In [ ]:
class BaseAgent:
    """Base agent class with common functionality"""
    def __init__(self, name: str, api_key: str = None):
        self.name = name
        self.api_key = api_key or os.getenv('OPENAI_API_KEY')
    
    def call_openai(self, system_prompt: str, user_prompt: str, mcp_context: Dict[str, Any] = None):
        """Call OpenAI API with system and user prompts using Model Context Protocol"""
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        # Create MCP formatted prompt if context is provided
        if mcp_context:
            mcp_header = f"""
                MODEL CONTEXT PROTOCOL
                AGENT: {mcp_context.get('agent', self.name)}
                TASK: {mcp_context.get('task', 'No task specified')}
                FORMAT: {mcp_context.get('format', 'json')}
"""
            # Add parameters section if present
            if 'parameters' in mcp_context and mcp_context['parameters']:
                params_str = "\n".join([f"- {k}: {v}" for k, v in mcp_context['parameters'].items()])
                mcp_header += f"PARAMETERS:\n{params_str}\n"
            
            user_prompt = f"{mcp_header}\n{user_prompt}"
        
        payload = {
            "model": "gpt-3.5-turbo",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        }
        
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            
            if response.status_code == 200:
                content = response.json()['choices'][0]['message']['content']
                return self._extract_json(content)
            return {"error": f"API Error: {response.status_code}"}
        except Exception as e:
            return {"error": str(e)}
    
    def _extract_json(self, content: str):
        """Extract JSON from text response"""
        try:
            return json.loads(content)
        except json.JSONDecodeError:
            # Try to extract JSON from text
            json_match = re.search(r'(\[.*\]|\{.*\})', content, re.DOTALL)
            if json_match:
                try:
                    return json.loads(json_match.group(1))
                except:
                    pass
            return {"error": "Failed to parse response"}


### 3. Flight Agent Class

The **FlightAgent** specializes in finding flight options:

- Purpose : Retrieves flight information using Aviation Stack API with OpenAI as fallback
- Key Methods :
  - **find_flights** : Main method that orchestrates flight search
  - **_get_aviation_stack_flights** : Calls Aviation Stack API for real flight data
  - **_calculate_duration** : Computes flight duration from departure and arrival times
  - **_generate_flight_data** : Uses OpenAI to generate realistic flight data when API fails
- API Integration : Prioritizes real flight data from Aviation Stack API
- Fallback Mechanism : Gracefully falls back to AI-generated data when API fails
- Data Processing : Transforms raw API responses into a consistent flight data format

In [12]:
class FlightAgent(BaseAgent):
    """Agent specialized in finding flights"""
    def __init__(self, api_key: str = None, aviation_api_key: str = None):
        super().__init__("Flight Agent", api_key)
        self.aviation_api_key = aviation_api_key or os.getenv('AVIATION_STACK_API_KEY')
    
    @mcp.tool(
        task_description="Find flight options from {origin} to {destination} on {date}"
    )
    def find_flights(self, origin: str, destination: str, date: str, mcp_context: Dict = None) -> List[Dict]:
        """Find flight options using Aviation Stack API if available"""
        # Try Aviation Stack API first
        if self.aviation_api_key:
            flights = self._get_aviation_stack_flights(origin, destination, date)
            if flights:
                return flights
        
        # Fallback to OpenAI
        return self._generate_flight_data(origin, destination, date, mcp_context)
    
    def _get_aviation_stack_flights(self, origin: str, destination: str, date: str) -> List[Dict]:
        """Get flights from Aviation Stack API"""
        try:
            url = "http://api.aviationstack.com/v1/flights"
            params = {
                'access_key': self.aviation_api_key,
                'dep_iata': origin,
                'arr_iata': destination,
                'flight_date': date
            }
            
            response = requests.get(url, params=params)
            
            if response.status_code == 200:
                data = response.json()
                flights = []
                
                for flight in data.get('data', [])[:3]:
                    departure = flight.get('departure', {})
                    arrival = flight.get('arrival', {})
                    
                    flights.append({
                        'airline': flight.get('airline', {}).get('name', 'Unknown'),
                        'flight_number': flight.get('flight', {}).get('iata', 'Unknown'),
                        'departure_time': departure.get('scheduled', 'Unknown'),
                        'arrival_time': arrival.get('scheduled', 'Unknown'),
                        'duration': self._calculate_duration(
                            departure.get('scheduled'),
                            arrival.get('scheduled')
                        ),
                        'from': departure.get('iata', origin),
                        'to': arrival.get('iata', destination),
                        'price': f"${random.randint(800, 1400)}"
                    })
                
                return flights
        except Exception:
            pass
        
        return []
    
    def _calculate_duration(self, departure_time, arrival_time):
        """Calculate flight duration from departure and arrival times"""
        if not departure_time or not arrival_time:
            return "N/A"
        
        try:
            dep = datetime.datetime.fromisoformat(departure_time.replace('Z', '+00:00'))
            arr = datetime.datetime.fromisoformat(arrival_time.replace('Z', '+00:00'))
            
            duration = arr - dep
            hours, remainder = divmod(duration.total_seconds(), 3600)
            minutes, _ = divmod(remainder, 60)
            
            return f"{int(hours)}h {int(minutes)}m"
        except Exception:
            return "N/A"
    
    def _generate_flight_data(self, origin: str, destination: str, date: str, mcp_context: Dict = None) -> List[Dict]:
        """Generate realistic flight data using OpenAI"""
        system_prompt = "You are a flight information specialist. Provide flight data in JSON format."
        user_prompt = f"""
        Generate 3 realistic flight options from {origin} to {destination} on {date}.
        Include airline name, flight number, departure time, arrival time, and duration.
        
        Return a valid JSON array of flight objects with these fields:
        - airline: string
        - flight_number: string
        - departure_time: ISO datetime string
        - arrival_time: ISO datetime string
        - duration: string (e.g. "11h 45m")
        - from: string (airport code)
        - to: string (airport code)
        - price: string (e.g. "$1,200")
        """
        
        result = self.call_openai(system_prompt, user_prompt, mcp_context)
        
        if isinstance(result, dict) and "flights" in result:
            return result["flights"]
        elif isinstance(result, list):
            return result
        return []

### 4. HotelAgent Class

The HotelAgent specializes in finding hotel accommodations:

- Purpose : Generates realistic hotel options for a given destination
- Key Methods :
  - **find_hotels** : Retrieves hotel information using OpenAI
- Prompt Engineering : Uses carefully crafted prompts to generate structured hotel data
- Data Format : Ensures consistent hotel data format with name, address, rating, price range, and amenities
- Response Processing : Handles different response formats from OpenAI API
- Fallback Behavior : Returns an empty list if no valid data is obtained

In [14]:
class HotelAgent(BaseAgent):
    """Agent specialized in finding hotels"""
    
    @mcp.tool(
        task_description="Find hotel options in {city}"
    )
    def find_hotels(self, city: str, mcp_context: Dict = None) -> List[Dict]:
        """Find hotel options using OpenAI"""
        system_prompt = "You are a hotel specialist. Provide hotel data in JSON format."
        user_prompt = f"""
        Generate 3 realistic hotel options in {city}.
        Include hotel name, address, rating (1-5), price range ($-$$$), and amenities.
        
        Return a valid JSON array of hotel objects with these fields:
        - name: string
        - address: string
        - rating: string (1-5)
        - price_range: string ($-$$$)
        - amenities: array of strings
        """
        
        result = self.call_openai(system_prompt, user_prompt, mcp_context)
        
        if isinstance(result, dict) and "hotels" in result:
            return result["hotels"]
        elif isinstance(result, list):
            return result
        return []

### 5. ItineraryAgent class

The ItineraryAgent creates detailed travel itineraries:

- Purpose : Generates day-by-day travel plans based on destination, duration, and selected flights/hotels
- Key Methods :
  - **create_itinerary** : Creates a detailed itinerary using OpenAI
  - **_create_fallback_itinerary** : Provides a backup itinerary when API fails
- Prompt Engineering : Uses detailed prompts that incorporate flight and hotel data
- Data Integration : Combines flight and hotel information into the itinerary planning
- Fallback Mechanism : Includes a robust fallback system with predefined activities
- Date Handling : Properly manages date progression throughout the itinerary
- Flexible Output : Supports both string-based and structured activity formats

In [15]:
class ItineraryAgent(BaseAgent):
    """Agent specialized in creating travel itineraries"""
    
    @mcp.tool(
        task_description="Create a {duration}-day itinerary for {destination} starting on {date}"
    )
    def create_itinerary(self, destination: str, duration: int, date: str, 
                         flights: List[Dict], hotels: List[Dict], mcp_context: Dict = None) -> Dict:
        """Create a travel itinerary using OpenAI"""
        system_prompt = "You are a travel planner who creates itineraries. Provide data in JSON format."
        user_prompt = f"""
        Create a {duration}-day itinerary for a trip to {destination} starting on {date}.
        
        Use these flight options: {json.dumps(flights[:1])}
        Use these hotel options: {json.dumps(hotels[:1])}
        
        Return a valid JSON object with:
        - trip_summary: string or object with trip details
        - daily_itinerary: array of day objects, each with:
          - date: string
          - activities: array of strings or objects with time, activity, and location
        """
        
        result = self.call_openai(system_prompt, user_prompt, mcp_context)
        
        if not result or "error" in result:
            return self._create_fallback_itinerary(destination, duration, date)
            
        return result
    
    def _create_fallback_itinerary(self, destination: str, duration: int, date: str) -> Dict:
        """Create a simple fallback itinerary when API fails"""
        try:
            start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
        except ValueError:
            start_date = datetime.datetime.now()
        
        itinerary = {
            "trip_summary": f"A {duration}-day trip to {destination}",
            "daily_itinerary": []
        }
        
        activities = [
            ["Breakfast at hotel", "City sightseeing", "Lunch at local restaurant", "Visit museums", "Dinner"],
            ["Breakfast", "Shopping tour", "Lunch", "Visit landmarks", "Dinner and nightlife"],
            ["Breakfast", "Day trip to nearby attractions", "Lunch", "Free time", "Farewell dinner"],
            ["Breakfast", "Cultural activities", "Lunch", "Relaxation time", "Local cuisine dinner"],
            ["Breakfast", "Souvenir shopping", "Lunch", "Airport transfer", "Return flight"]
        ]
        
        for i in range(duration):
            current_date = start_date + datetime.timedelta(days=i)
            date_str = current_date.strftime("%Y-%m-%d")
            
            itinerary["daily_itinerary"].append({
                "date": date_str,
                "activities": activities[i % len(activities)]
            })
        
        return itinerary

### 6. TravelPlanner Class

The TravelPlanner coordinates all agents and manages the overall trip planning process:

- Purpose : Orchestrates the entire trip planning workflow and displays results
- Key Methods :
  - __init__ : Initializes agents and output file
  - **log** : Writes messages to both console and output file
  - **plan_trip** : Main method that coordinates the entire trip planning process
  - **display_trip_plan** : Formats and displays the trip plan in a readable format
- Agent Coordination : Manages the workflow between specialized agents
- Output Management : Handles both console output and file logging
- Data Aggregation : Combines results from all agents into a comprehensive trip plan
- Display Formatting : Uses emojis and structured formatting for readable output
- Error Resilience : Continues operation even if individual components fail

In [16]:
class TravelPlanner:
    def __init__(self):
        # Get API keys from environment variables
        self.api_key = os.getenv('OPENAI_API_KEY')
        self.aviation_api_key = os.getenv('AVIATION_STACK_API_KEY')
        
        # Initialize specialized agents
        self.flight_agent = FlightAgent(self.api_key, self.aviation_api_key)
        self.hotel_agent = HotelAgent(self.api_key)
        self.itinerary_agent = ItineraryAgent(self.api_key)
        
        # Setup output file
        self.output_file = "c:\\Users\\Omkar\\PROJECTS\\Langchain_projects\\AgenticAI\\AgentSDK_Tutorials\\advanced_agents\\output.txt"
        with open(self.output_file, 'w', encoding='utf-8') as f:
            f.write("Travel Planner Output\n")
            f.write("====================\n\n")
    
    def log(self, message):
        """Write message to both console and output file"""
        print(message)
        with open(self.output_file, 'a', encoding='utf-8') as f:
            f.write(message + "\n")
    
    def plan_trip(self, origin: str, destination: str, date: str, duration: int = 3):
        """Plan a complete trip"""
        self.log(f"Planning a {duration}-day trip to {destination}...")
        
        # Get flights, hotels, and create itinerary
        self.log("Finding flights...")
        flights = self.flight_agent.find_flights(origin, destination, date)
        
        self.log("Finding hotels...")
        hotels = self.hotel_agent.find_hotels(destination)
        
        self.log("Creating itinerary...")
        itinerary = self.itinerary_agent.create_itinerary(destination, duration, date, flights, hotels)
        
        # Display the trip plan
        self.display_trip_plan(flights, hotels, itinerary)
        
        self.log("\nTrip planning complete! Full details saved to output.txt")
        
        return {
            "flights": flights,
            "hotels": hotels,
            "itinerary": itinerary
        }
    
    def display_trip_plan(self, flights: List[Dict], hotels: List[Dict], itinerary: Dict):
        """Display the trip plan in a readable format"""
        # [Rest of the method unchanged]
        # Display flights
        self.log("\n🛫 Flight Options:")
        for i, flight in enumerate(flights, 1):
            self.log(f"{i}. {flight.get('airline', 'Unknown')} {flight.get('flight_number', 'Unknown')}")
            self.log(f"   From: {flight.get('from', 'Unknown')} at {flight.get('departure_time', 'Unknown')}")
            self.log(f"   To: {flight.get('to', 'Unknown')} at {flight.get('arrival_time', 'Unknown')}")
            self.log(f"   Duration: {flight.get('duration', 'N/A')}")
            if 'price' in flight:
                self.log(f"   Price: {flight.get('price', 'N/A')}")
            self.log("")
        
        # Display hotels
        self.log("\n🏨 Hotel Options:")
        for i, hotel in enumerate(hotels, 1):
            self.log(f"{i}. {hotel.get('name', 'Unknown Hotel')} ({hotel.get('rating', 'N/A')}★)")
            self.log(f"   Address: {hotel.get('address', 'N/A')}")
            self.log(f"   Price: {hotel.get('price_range', 'N/A')}")
            if 'amenities' in hotel and isinstance(hotel['amenities'], list):
                self.log(f"   Amenities: {', '.join(hotel['amenities'])}")
            self.log("")
        
        # Display itinerary
        self.log("\n📅 Trip Itinerary:")
        
        # Display summary
        summary = itinerary.get('trip_summary', 'No summary available')
        if isinstance(summary, dict):
            self.log("Trip Summary:")
            for key, value in summary.items():
                if not isinstance(value, (dict, list)):
                    self.log(f"  {key.replace('_', ' ').title()}: {value}")
        else:
            self.log(f"Trip Summary: {summary}")
        
        # Display daily itinerary
        daily = itinerary.get('daily_itinerary', []) or itinerary.get('days', [])
        
        for i, day in enumerate(daily, 1):
            date = day.get('date', f"Day {i}")
            self.log(f"\nDay {i} - {date}:")
            
            activities = day.get('activities', [])
            for activity in activities:
                if isinstance(activity, dict):
                    time = activity.get('time', '')
                    desc = activity.get('activity', activity.get('description', ''))
                    loc = activity.get('location', '')
                    
                    self.log(f"  {time} - {desc}")
                    if loc:
                        self.log(f"    Location: {loc}")
                elif isinstance(activity, str):
                    self.log(f"  • {activity}")

In [17]:
# main function
def main():
    planner = TravelPlanner()
    planner.plan_trip(
        origin="SFO",
        destination="Tokyo",
        date="2024-09-15",
        duration=5
    )
    print(f"Complete trip details have been saved to: {planner.output_file}") # saves the output in "output.txt"

if __name__ == "__main__":
    main()

Planning a 5-day trip to Tokyo...
Finding flights...
Finding hotels...
Creating itinerary...

🛫 Flight Options:
1. United Airlines UA837
   From: SFO at 2024-09-15T08:00:00Z
   To: HND at 2024-09-16T12:45:00Z
   Duration: 14h 45m
   Price: $1,200

2. ANA - All Nippon Airways NH7
   From: SFO at 2024-09-15T10:30:00Z
   To: NRT at 2024-09-16T15:15:00Z
   Duration: 13h 45m
   Price: $1,350

3. Singapore Airlines SQ2
   From: SFO at 2024-09-15T13:45:00Z
   To: HND at 2024-09-16T18:55:00Z
   Duration: 13h 10m
   Price: $1,100


🏨 Hotel Options:
1. Imperial Hotel Tokyo (5★)
   Address: 1 Chome-1-1 Uchisaiwai-cho, Chiyoda City, Tokyo 100-8558, Japan
   Price: $$$
   Amenities: Spa, Fitness Center, Restaurants, Concierge Service

2. Park Hyatt Tokyo (4★)
   Address: 3-7-1-2 Nishishinjuku, Shinjuku City, Tokyo 163-1055, Japan
   Price: $$$$
   Amenities: Swimming Pool, Spa, Restaurant, Room Service

3. Andaz Tokyo Toranomon Hills (4★)
   Address: 1-23-4 Toranomon, Minato City, Tokyo 105-0001, J